In [63]:
import requests
import pandas as pd
from io import StringIO
pd.options.display.max_colwidth = 2000

In [64]:
df=pd.read_excel('빅카인즈_뉴스.xlsx')
df=df.loc[:,['일자','제목','본문']]

C:\Users\82103\anaconda3\envs\DL\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [65]:
#중복 제거
kr_news = df.drop_duplicates(subset=['본문'],keep='first').reset_index(drop=True)
print(kr_news.shape)

(16846, 3)


In [66]:
# 테스트를 위해 3000개만..

kr_df=kr_news.iloc[:3000,:]

In [67]:
#문장 단위로 짜르기 위함.
from koalanlp.Util import initialize, finalize
from koalanlp.proc import SentenceSplitter
from koalanlp import API

initialize(OKT='LATEST')  #: HNN=2.0.3

splitter = SentenceSplitter(API.OKT)


Exception: JVM cannot be initialized more than once.Please call koalanlp.Util.finalize() when you want to re-init the JVM with other options.

In [41]:
kr_df.본문[0]

' 아시아경제 손선희 기자 지난해 부진한 실적을 기록한 현대백화점에 대해 증권사들이 줄줄이 목표주가를 낮춰 잡았다 한국투자증권은 14일 현대백화점에 대한 목표주가를 기존 9만원에서 8만원으로 하향 조정한다고 밝혔다 현대백화점은 지난해 4분기 연결기준 영업이익 686억원을 기록했다 전년 동기 대비 27 2 줄었을 뿐만 아니라 시장 기대치에도 '

In [68]:

from tqdm.auto import tqdm
sentence_list=[]
index_list=[]
for i in tqdm(range(kr_df.shape[0])):
    s_text=splitter(kr_df.본문[i])
    sentence_list.append(s_text)
    count=len(s_text)
    index_list.append(count)
    
    

  0%|          | 0/3000 [00:00<?, ?it/s]

In [69]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tag2id = {'B-ORG': 0, 'O': 1, 'I-ORG': 2}
unique_tags={'B-ORG', 'I-ORG', 'O'}
id2tag={0: 'B-ORG', 1: 'O', 2: 'I-ORG'}
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

model = AutoModelForTokenClassification.from_pretrained('kcelectra_base_new', num_labels=len(unique_tags))
model.to(device)

ElectraForTokenClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [70]:
# 기존 토크나이저는 wordPiece tokenizer로 tokenizing 결과를 반환합니다.
# 데이터 단위를 음절 단위로 변경했기 때문에, tokenizer도 음절 tokenizer로 변경

# berttokenizer를 사용하는데 한국어 vocab이 8000개 정도 밖에 없고 그 안의 한국어들의 거의 음절로 존재
# -> 음절 단위 tokenizer를 적용하면 vocab id를 어느 정도 획득할 수 있어 UNK가 별로 없을듯 하다
def ner_tokenizer(sent, max_seq_length):    
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1
    
    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [71]:
# 우리가 전에 사용했던건 word piece tokenizer
# 지금 사용한건 음절단위 tokenizer
# 반드시 음절 tokenizer를 거친 후에 model에 들어가야 한다.

def ner_inference(text) : 
    words = []
    tags= []
    model.eval()
    text = text.replace(' ', '_')

    predictions, true_labels = [], []
    
    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)    
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    for i, tag in enumerate(pred_tags):
      words.append(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]))
      tags.append(tag)
    return words, tags




In [72]:
# 사용 환경에 맞게 후 처리
index_list=[]
text_list= []
outputs_list=[]
tags_list =[]

for i in tqdm(range(len(sentence_list))):
    for text in sentence_list[i]:
        words,tags = ner_inference(text)
        index_list.append(i)

        text_list.append(text)
        tags_list.append(tags)
        outputs_list.append(words)

df=pd.DataFrame(zip(index_list,text_list,outputs_list,tags_list))
df.columns=['news_index','text','output','tag']

sum_list=[]
for i in range(df.news_index.nunique()):
    s_text=''.join(df[df.loc[:,'news_index']==i].text.to_list())
    count = sum(df.loc[:,'news_index']==i)
    for j in range(count):
        sum_list.append(s_text)
df['sum_text']=sum_list





  0%|          | 0/3000 [00:00<?, ?it/s]

In [73]:
import pandas as pd
import ast


def export(test_tag, test_text):

    # b index list
    b_list = [i for i, x in enumerate(test_tag) if x == "B-ORG"]

    # 임시 i list (나중에 b list랑 더함)
    ii_list = []
    for n in range(len(b_list)):
        if n + 1 < len(b_list):
            sample = test_tag[(b_list[n]):(b_list[n+1])]
        else:
            sample = test_tag[(b_list[n]):]

        sample.reverse()

        if "I-ORG" in sample:
            ii_list.append(len(sample) - sample.index("I-ORG"))
        else:
            ii_list.append(0)


    # i list만들기 (임시 i list + b list)
    i_list = []
    for i, j in zip(b_list, ii_list):
        i_list.append(i + j)


    # b, i 매칭 -> 튜플로
    ticker_range = list(zip(b_list, i_list))

    # 티커 리스트
    ticker_list = []

    for r in ticker_range:
        ticker_list.append(test_text[r[0]-1:r[1]-1])

    return ticker_list



In [74]:
df["기업명"] = df.apply(lambda x: export(x.tag, x.text), axis=1)
df.기업명=df.기업명.apply(lambda x : ','.join(x))

In [43]:
df.to_csv('한국뉴스ner_모델.csv',encoding='utf-8-sig')

In [299]:
# last_comp_list=[]


# for i in range(len(df)):
#   lst = df['tag'][i]
#   b = np.array(lst)
#   end = 0
#   comp_list=[]
#   for j in range(len(np.where(b == 'B-ORG')[0])):
#     start = np.where(b == 'B-ORG')[0][j] - 1
#     end_list = list(np.where(b == 'I-ORG')[0])
#     end_list.append(999)
    
#     for k in range(end+1, len(end_list)):
#         if end_list[k] - end_list[k-1] != 1:
#             end = k
#             break
#         else:
#             continue
#     comp=df['text'][i][start:end_list[end-1]]
#     comp_list.append(comp)
#   last_comp_list.append(comp_list)

# df['tickers']=last_comp_list
# ################################
# def tag_overlap(tag,indices):
#     stacks = []
#     for idx, oov in enumerate(indices['O']):
#         if idx < len(indices['O'])-1:
#             # Not B-ORG -> stack push
#             if (indices['O'][idx+1] - indices['O'][idx] == 1 or tag[indices['O'][idx]+1] == 'I-ORG'):
#                 stacks.append(oov)
#             # Begin -> reset
#             if tag[indices['O'][idx]+1] == 'B-ORG':
#                 stacks = []
#             # Abnormal I-ORG reached -> OOV to I-ORG
#             elif tag[indices['O'][idx]+1] == 'I-ORG':
#                 indices['I-ORG'].extend(stacks)
#                 stacks = []
#     indices['I-ORG'] = sorted(indices['I-ORG'])
    
#     return indices
# #############################################
# comp_list=[]
# for i in tqdm(range(len(df))):
# # i = 1314
#     tickers = []

#     lst = df['tag'][i]

#     # Initialize dict of indices
#     indices = dict.fromkeys(lst,[])
#     indices = {tag:list(np.where(np.array(lst)==tag)[0]) for tag in indices}


#     try:
#         # Tag operation
#         indices = tag_overlap(lst,indices)
#         start_idx = indices['B-ORG']
#         end_idx = indices['I-ORG']
#         end_start_diff = [end_idx[idx]-end_idx[idx-1] if idx>0 else 0 for idx in range(len(end_idx))]

# #         print(indices)
# #         print(end_start_diff)



#         #     diff_idx = [diff for diff in end_start_diff if diff != 1][:len(start_idx)]
#         diff_idx = np.where(np.array(end_start_diff)!=1)[0][:len(start_idx)]

#         indexes = [(start_idx[idx]-1,start_idx[idx]+(diff_idx[idx+1]-diff_idx[idx])) if idx < len(diff_idx)-1 else (start_idx[idx]-1,end_idx[-1]) for idx in range(len(diff_idx))]

#         tickers = [df['text'][i][tup[0]:tup[1]] for tup in indexes]

#     except:
#         pass

#     comp_list.append(tickers)

# #     start_idx = np.where(np.array(lst) == 'B-ORG')[0]

# #     if len(start_idx) == 0:
# #         comp_list.append(tickers)
# #         continue

# #     end_idx = np.where(np.array(lst) == 'I-ORG')[0]

# #     end_start_diff = [end_idx[idx]-end_idx[idx-1] if idx>0 else 0 for idx in range(len(end_idx))]
# # #     diff_idx = [diff for diff in end_start_diff if diff != 1][:len(start_idx)]
# #     diff_idx = np.where(np.array(end_start_diff)!=1)[0][:len(start_idx)]

# #     indexes = [(start_idx[idx]-1,start_idx[idx]+(diff_idx[idx+1]-diff_idx[idx])) if idx < len(diff_idx)-1 else (start_idx[idx]-1,end_idx[-1]) for idx in range(len(diff_idx))]

# #     tickers = [df['text'][i][tup[0]:tup[1]] for tup in indexes]
# #     comp_list.append(tickers)

# comp_list


In [76]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [77]:
import torch.cuda.amp as amp
import os
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification,\
XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)


# class args
class args:
    # ---- factor ---- #
    debug=False
    amp = True
    gpu = '0'
    
    batch_size=32
    max_len = 128
    
    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None#'get_linear_schedule_with_warmup'

In [78]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base", cache_dir='bert_ckpt', do_lower_case=False)

In [79]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'text']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = args.max_len,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

test_data = TestDataset(df)
    
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=args.batch_size,
                                        shuffle=False,
                                      num_workers=0)

In [80]:






def do_predict(net, valid_loader):
    
    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)[0]

            else:
                output = net(outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask))
             
            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())
            
    return pred_lst,logit

def run_predict(model_path,test_dataloader):


    print('set testloader')
    ## net ----------------------------------------
    scaler = amp.GradScaler()

    net = RobertaForSequenceClassification.from_pretrained('klue/roberta-base', num_labels = 3) 

        
    net.to(device)
    
    if len(args.gpu)>1:
        net = nn.DataParallel(net)

    f = torch.load(model_path)
    net.load_state_dict(f, strict=True)  # True
    print('load saved models')
    # ------------------------
    # validation
    preds, logit = do_predict(net, test_dataloader) #outputs
           
    print('complete predict')
    
    return preds, np.array(logit)

In [81]:
preds1, logit1 = run_predict('./model/klue_base_fold3_s.pth',test_dataloader)

set testloader


Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifie

load saved models


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15640\2994675733.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):


  0%|          | 0/311 [00:00<?, ?it/s]

C:\Users\82103\anaconda3\envs\DL\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


complete predict


In [82]:
df['감성분류']=preds1
df.감성분류=df.감성분류.apply(lambda x: '중립' if x == 0 else '긍정' if x == 1 else '부정')

In [83]:
last_=df.loc[:,['text','tag','기업명','감성분류']]

In [84]:
last_[:60]

,text,tag,기업명,감성분류
0,[아시아경제 손선희 기자] 지난해 부진한 실적을 기록한 현대백화점에 대해 증권사들이 줄줄이 목표주가를 낮춰 잡았다.,"[O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","아시아경제,현대백화점",부정
1,한국투자증권은 14일 현대백화점에 대한 목표주가를 기존 9만원에서 8만원으로 하향 조정한다고 밝혔다.,"[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","한국투자증권,현대백화점",부정
2,현대백화점은 지난해 4분기 연결기준 영업이익 686억원을 기록했다.,"[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",현대백화점,중립
3,전년 동기 대비 27.2% 줄었을 뿐만 아니라 시장 기대치에도 ..,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",,부정
4,[헤럴드경제=증권부] 필옵틱스는 지난해 연결기준 영업이익이 176억9866만원으로 전년대비 흑자전환했다고 14일 공시했다.,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",필옵틱스,긍정
5,같은 기간 매출액은 3039억8773만원으로 31.7% 증가한 것으로 나타났다.,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",,긍정
6,당기순이익 역시 55억2147만원으로 흑자전환했다.,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",,긍정
7,jiyoon436@heraldcorp.com,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",,중립
8,"펄어비스가 지난해 매출 3860억원, 영업이익 166억원, 당기순손실 411억원을 기록했다고 14일 밝혔다.","[O, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",펄어비스,중립
9,"전년 대비 매출과 영업이익이 각각 4.4%, 61.4% 감소했다.","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",,부정


In [85]:
last_.to_csv('test_ner_sentiment.csv',encoding='utf-8-sig')